In [1]:
import nltk 
nltk.download("gutenberg")
from  nltk.corpus import gutenberg
import pandas as pd

data = gutenberg.raw("shakespeare-hamlet.txt")

with open("hamlet.txt", "w") as f:
    f.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
with open("hamlet.txt") as f:
    data = f.read().lower()


In [ ]:
from datasets import load_dataset
data = load_dataset('fdaudens/hf-blog-posts')

In [ ]:
text = ''.join(str([data['train'][i]['Headline'] + " " + str(data['train'][i]['Article Text']) for i in range(len(data))]))

In [ ]:
with open("blog.txt", "w") as f:
    f.write(text)


In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [5]:
# tokenizer.word_index

In [6]:
inputseq = [] 
for line in data.split("\n"):
    tokenlist = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokenlist)):
        n_gram_sequence = tokenlist[:i+1]
        inputseq.append(n_gram_sequence)

In [7]:
inputseq

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [8]:
maxseq = max([len(x) for x in inputseq])
maxseq

14

In [9]:
inputseq = np.array(pad_sequences(inputseq, maxlen=maxseq, padding="pre"))

In [10]:
inputseq

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [11]:
import tensorflow as tf
x, y = inputseq[:,:-1], inputseq[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)  

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [15]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=maxseq+1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(total_words, activation="softmax")) # ouput is multiclass classification so softmax

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

c:\Users\LENOVO\Desktop\ML_Projects\Machine-Learning\LSTM_Next_Word\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), verbose=1)

Epoch 1/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.0342 - loss: 7.1121 - val_accuracy: 0.0336 - val_loss: 6.6768
Epoch 2/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.0399 - loss: 6.4452 - val_accuracy: 0.0412 - val_loss: 6.7342
Epoch 3/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.0432 - loss: 6.2962 - val_accuracy: 0.0499 - val_loss: 6.7686
Epoch 4/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0507 - loss: 6.1468 - val_accuracy: 0.0552 - val_loss: 6.7987
Epoch 5/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0581 - loss: 5.9920 - val_accuracy: 0.0598 - val_loss: 6.8517
Epoch 6/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.0676 - loss: 5.8071 - val_accuracy: 0.0577 - val_loss: 6.8899
Epoch 7/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0746 - loss: 5.6533 - val_accuracy: 0.0672 - val_loss: 6.9832
Epoch 8/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0810 - loss: 5

In [17]:
def predict_next_word(model, tokenizer, seed_text, maxseq):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    if len(token_list) >= maxseq:
        token_list = token_list[-maxseq-1:]
    token_list = pad_sequences([token_list], maxlen=maxseq-1, padding="pre")
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word 
    return None

In [22]:
input_text = "This is an red fruit called "
print(f"Input text: {input_text}")
max_sq_len = model.input_shape[1]+1
next_words = predict_next_word(model, tokenizer, input_text, max_sq_len)
print(f"Next word: {next_words}")

Input text: This is an red fruit called 
Next word: growes


In [21]:
import time
text = 'This is an red fruit called '  ## initial word
    # tokenize 
token_text = tokenizer.texts_to_sequences([text])[0]
# padding
padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
# predict 
pos = np.argmax(model.predict(padded_token_text))  ## predicting new word

for word, index in tokenizer.word_index.items():
    if index == pos:
        text = text + " " + word
        print(text)
        # time.sleep(0./5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
This is an red fruit called  phrase


In [20]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)